In [3]:
import pandas as pd
from geopy.distance import geodesic
from scipy.stats import spearmanr
import geopandas as gpd

from shapely.geometry import Polygon, Point
import pickle

In [4]:
with open('../data/manipulated/buildings.pkl', 'rb') as f:
    building = pickle.load(f)

In [3]:
def polygon_to_point(polygon):
    centroid = polygon.centroid
    latitude = centroid.y
    longitude = centroid.x
    return Point(longitude, latitude)

def point_to_coords(point):
    latitude = point.y
    longitude = point.x
    return (latitude, longitude)


building['centroid'] = building['geometry'].apply(polygon_to_point)
building['latitude'], building['longitude'] = zip(*building['centroid'].map(point_to_coords))

In [4]:
building.head()

,obj_type,name,info,damage_gra,notation,or_src_id,cd_value,geometry,Province,city,...,labour_fource,unemployment,agricultural,life_time,hb_per100000,fertility,hh_size,centroid,latitude,longitude
0,11-Residential Buildings,Unknown,997-Not Applicable,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31461 37.76919, 38.31480 37.76871...",ADIYAMAN,ADIYAMAN,...,50.0,10.1,2336012,79.7,222,2.36,4.0,POINT (38.31425 37.76887),37.768867,38.314252
1,11-Residential Buildings,Unknown,997-Not Applicable,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31327 37.76841, 38.31297 37.76834...",ADIYAMAN,ADIYAMAN,...,50.0,10.1,2336012,79.7,222,2.36,4.0,POINT (38.31328 37.76869),37.768690,38.313284
2,12-Non-residential Buildings,Unknown,1251-Industrial buildings,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31832 37.76898, 38.31826 37.76822...",ADIYAMAN,ADIYAMAN,...,50.0,10.1,2336012,79.7,222,2.36,4.0,POINT (38.31801 37.76861),37.768612,38.318013
3,12-Non-residential Buildings,Unknown,1251-Industrial buildings,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31898 37.76892, 38.31893 37.76819...",ADIYAMAN,ADIYAMAN,...,50.0,10.1,2336012,79.7,222,2.36,4.0,POINT (38.31868 37.76857),37.768575,38.318677
4,11-Residential Buildings,Unknown,997-Not Applicable,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31286 37.76831, 38.31243 37.76819...",ADIYAMAN,ADIYAMAN,...,50.0,10.1,2336012,79.7,222,2.36,4.0,POINT (38.31254 37.76851),37.768514,38.312542


In [5]:
fault = gpd.read_file('../data/Fault_shp/Diri Faylar.shp')
fault['geometry'] = fault['geometry'].to_crs(epsg=4326)
fault['centroid'] = fault['geometry'].apply(polygon_to_point)
fault['latitude'], fault['longitude'] = zip(*fault['centroid'].map(point_to_coords))

fault.head()

,Id,ACIKLAMA,geometry,centroid,latitude,longitude
0,0,Bindirme,"LINESTRING (39.28717 38.24507, 39.28811 38.245...",POINT (39.57409 38.28385),38.283854,39.574094
1,2,Bindirme,"LINESTRING (40.69298 38.47101, 40.69322 38.471...",POINT (40.73709 38.49215),38.492145,40.737086
2,1,Bindirme,"LINESTRING (40.55238 38.47913, 40.55370 38.477...",POINT (40.62358 38.47362),38.473618,40.623575
3,0,Bindirme,"MULTILINESTRING ((40.69298 38.47101, 40.69322 ...",POINT (40.67083 38.48133),38.481332,40.670832
4,0,Bindirme,"MULTILINESTRING ((40.69285 38.47068, 40.69154 ...",POINT (40.67083 38.48133),38.481332,40.670832


In [7]:
from geopy.distance import geodesic

# Iterate over each building in the 'building_damage_data' dataframe
for index, row in building.iterrows():
    # Get the latitude and longitude of the building
    lat = row['latitude']
    lon = row['longitude']
    
    # Calculate the distance to the nearest fault using geodesic distance
    distances = []
    for index2, row2 in fault.iterrows():
        fault_lat = row2['latitude']
        fault_lon = row2['longitude']
        distances.append(geodesic((lat, lon), (fault_lat, fault_lon)).km)
    nearest_fault_distance = min(distances)
    
    # Store the distance in a new column of the 'building_damage_data' dataframe
    building.at[index, 'nearest_fault_distance_km'] = nearest_fault_distance

building.head()


,obj_type,name,info,damage_gra,notation,or_src_id,cd_value,geometry,Province,city,...,unemployment,agricultural,life_time,hb_per100000,fertility,hh_size,centroid,latitude,longitude,nearest_fault_distance_km
0,11-Residential Buildings,Unknown,997-Not Applicable,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31461 37.76919, 38.31480 37.76871...",ADIYAMAN,ADIYAMAN,...,10.1,2336012,79.7,222,2.36,4.0,POINT (38.31425 37.76887),37.768867,38.314252,2.766849
1,11-Residential Buildings,Unknown,997-Not Applicable,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31327 37.76841, 38.31297 37.76834...",ADIYAMAN,ADIYAMAN,...,10.1,2336012,79.7,222,2.36,4.0,POINT (38.31328 37.76869),37.768690,38.313284,4.076484
2,12-Non-residential Buildings,Unknown,1251-Industrial buildings,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31832 37.76898, 38.31826 37.76822...",ADIYAMAN,ADIYAMAN,...,10.1,2336012,79.7,222,2.36,4.0,POINT (38.31801 37.76861),37.768612,38.318013,4.037559
3,12-Non-residential Buildings,Unknown,1251-Industrial buildings,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31898 37.76892, 38.31893 37.76819...",ADIYAMAN,ADIYAMAN,...,10.1,2336012,79.7,222,2.36,4.0,POINT (38.31868 37.76857),37.768575,38.318677,3.995659
4,11-Residential Buildings,Unknown,997-Not Applicable,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31286 37.76831, 38.31243 37.76819...",ADIYAMAN,ADIYAMAN,...,10.1,2336012,79.7,222,2.36,4.0,POINT (38.31254 37.76851),37.768514,38.312542,4.015685


In [ ]:
building.head()

In [8]:
building.to_pickle('../data/manipulated/building1.pickle')

In [16]:
# in order not to lose the data we are saving to pickle file

# Opening  the pickle file again
with open('../data/manipulated/building1.pickle', 'rb') as file:
    # Load the object from the pickle file
    building_w_distance = pickle.load(file)
    
building_w_distance.columns

Index(['obj_type', 'name', 'info', 'damage_gra', 'notation', 'or_src_id',
       'cd_value', 'geometry', 'Province', 'city', 'population', 'income',
       'total_sales', 'second_sales', 'water_access', 'elec_cons',
       'building_perm', 'land_permited', 'labour_fource', 'unemployment',
       'agricultural', 'life_time', 'hb_per100000', 'fertility', 'hh_size',
       'centroid', 'latitude', 'longitude', 'nearest_fault_distance_km'],
      dtype='object')

In [17]:
with open('../data/manipulated/day_eq.pkl', 'rb') as file2:
    # Load the object from the pickle file
    day_eq = pickle.load(file2)
    
day_eq.columns

Index(['Date', 'Longitude', 'Latitude', 'Depth', 'Rms', 'Type', 'Magnitude',
       'Location', 'EventID', 'Province', 'Municipio', 'Municipio-Province'],
      dtype='object')

In [18]:
    
    
# Iterate over each building in the 'building_damage_data' dataframe
for index, row in building_w_distance.iterrows():
    # Get the latitude and longitude of the building
    lat = row['latitude']
    lon = row['longitude']
    
    # Calculate the distance to the nearest fault using geodesic distance
    distances = []
    for index2, row2 in day_eq.iterrows():
        eq_lat = row2['Latitude']
        eq_lon = row2['Longitude']
        distances.append(geodesic((lat, lon), (eq_lat, eq_lon)).km)
    nearest_eq_distance = min(distances)
    
    # Store the distance in a new column of the 'building_damage_data' dataframe
    building_w_distance.at[index, 'nearest_eq_distance_km'] = nearest_eq_distance

building_w_distance.head()

,obj_type,name,info,damage_gra,notation,or_src_id,cd_value,geometry,Province,city,...,agricultural,life_time,hb_per100000,fertility,hh_size,centroid,latitude,longitude,nearest_fault_distance_km,nearest_eq_distance_km
0,11-Residential Buildings,Unknown,997-Not Applicable,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31461 37.76919, 38.31480 37.76871...",ADIYAMAN,ADIYAMAN,...,2336012,79.7,222,2.36,4.0,POINT (38.31425 37.76887),37.768867,38.314252,2.766849,9.630984
1,11-Residential Buildings,Unknown,997-Not Applicable,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31327 37.76841, 38.31297 37.76834...",ADIYAMAN,ADIYAMAN,...,2336012,79.7,222,2.36,4.0,POINT (38.31328 37.76869),37.768690,38.313284,4.076484,10.449613
2,12-Non-residential Buildings,Unknown,1251-Industrial buildings,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31832 37.76898, 38.31826 37.76822...",ADIYAMAN,ADIYAMAN,...,2336012,79.7,222,2.36,4.0,POINT (38.31801 37.76861),37.768612,38.318013,4.037559,10.466301
3,12-Non-residential Buildings,Unknown,1251-Industrial buildings,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31898 37.76892, 38.31893 37.76819...",ADIYAMAN,ADIYAMAN,...,2336012,79.7,222,2.36,4.0,POINT (38.31868 37.76857),37.768575,38.318677,3.995659,10.480286
4,11-Residential Buildings,Unknown,997-Not Applicable,No visible damage,Building block,1,Not Applicable,"POLYGON ((38.31286 37.76831, 38.31243 37.76819...",ADIYAMAN,ADIYAMAN,...,2336012,79.7,222,2.36,4.0,POINT (38.31254 37.76851),37.768514,38.312542,4.015685,10.569504


In [1]:
building_w_distance.to_pickle('../data/manipulated/building_eq_distance.pkl')



NameError: name 'building_w_distance' is not defined

In [6]:
building_w_distance = pd.read_pickle('../data/manipulated/building_eq_distance.pkl')

socioeconomic = pd.read_pickle('../data/manipulated/merged_data.pkl')

In [14]:
data_final = building_w_distance.merge(socioeconomic, on='Province', how='left')
data_final.head()

data_final.to_pickle('../data/manipulated/data_final.pkl')


: 